In [2]:
pip install opencv-python easyocr

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
torch.cuda.is_available()

True

In [ ]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw

In [ ]:
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    # Use adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Dilate the image to connect individual elements
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=1)
    
    return dilate

In [ ]:
def run_model(image_path):
# 이미지 경로


# OpenCV를 사용하여 이미지 읽기

    image = image_path

# EasyOCR을 초기화하고 사용할 언어를 설정
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

# 이미지에서 텍스트 추출
    result = reader.readtext(image)
    
    for (bbox, text, prob) in result:
        print(f'{text}')

    return None

In [289]:
def run_boxModel(image_path):
    import easyocr
    import cv2

# Create an EasyOCR reader
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

    proccessed_iamge = preprocess_image(image_path)

# Detect text and bounding boxes
    results = reader.readtext(proccessed_iamge, width_ths=50000, ycenter_ths = 1)

    image = cv2.imread(image_path)

# Detect text and bounding boxes
    # kernel = np.ones((5,5), np.uint8)
    # # dilate = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    # dilate = cv2.erode(image, kernel)

    cv2.imwrite('train/redbox_image/dilate_image.jpg', image)
    lists = []
    for i, (bbox, text, prob) in enumerate(results):
    # Extract coordinates of the bounding box
        top_left = (int(bbox[0][0]), int(bbox[0][1]))
        bottom_right = (int(bbox[2][0]), int(bbox[2][1]))

    # Crop the bounding box area
        cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]


        crop_results = reader.recognize(cropped_image)

        for (bbox, text, prob) in crop_results:
            # print(text)
            lists.append(text)
            
        
    # Save the cropped image
        Image.fromarray(cropped_image).save(f'cropped_text_{i}.png')
    flag = False    
    for (bbox, text, prob) in results:
        if "판매소계" in str(text):
            break
        if flag:
            print(text)
        if "품명" or "단가" or "수량" or "금액" in text:
            flag = True


    
        #print(f"Text: {text}, Confidence: {prob}")
    return None

In [225]:
def run_redboxModel(image_path):
    import easyocr
    import cv2
# Create an EasyOCR reader
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

    proccessed_image = preprocess_image(image_path)

# Detect text and bounding boxes
    # results = reader.detect(proccessed_image, width_ths=0.7)
    # result = results[0]
    image = cv2.imread(image_path)

    result = reader.readtext(proccessed_image, width_ths = 500, ycenter_ths = 1)
    
    for i, (bbox, text, prob) in enumerate(result):
    # Extract coordinates of the bounding box
        top_left = (int(bbox[0][0]), int(bbox[0][1]))
        bottom_right = (int(bbox[2][0]), int(bbox[2][1]))
        
    # #Unpack the bounding box
    #     (top_left, top_right, bottom_right, bottom_left) = bbox

    # # Convert coordinates to integer
    #     top_left = (int(top_left[0]), int(top_left[1]))
    #     bottom_right = (int(bottom_right[0]), int(bottom_right[1]))

    # Draw a red rectangle on the original image
        cv2.rectangle(image, top_left, bottom_right, (0, 0, 255), 2)  # Red color and 2 px thickness

# Display the image with bounding boxes
    cv2.imwrite('train/redbox_image/image_with_boxes.jpg', image)
    return None

In [290]:
image_path = '/Users/402-08/train/img19.jpg'
run_boxModel(image_path)

뭐해
0
디(D)
'청평화 지하 가
H:P: 0jo-48gg-8070
Nalauspin-BGOf-ZnGpaainP0s0o
'KaKaoTaik ID: dodreaii2o5o
maEhat ID;' dadraamzoao
신장마켓  ID; 데디(DgD)-
22-02(원) ;합 500 @oo  영수
출명   배 ' 단가 ' 수량: : 하   곰액
비비프레배기그러이j6,000  6 120,000
비비프렉배키 그리이28 000 a g 실  60-0003
비비표켓배거 흐레이씨
2O,000:` 6;    lZO;OoDO=
비표a배기베이지& 0003 9하다 다다 80,000
비비포키배기| 그테이 8 000 - _ 8  " b0,000
비비포기버기 그래o0 000  3560,00:


In [226]:
image_path = '/Users/402-08/train/img73.jpg'
run_redboxModel(image_path)

In [222]:
def slice_image(image_path):
    import cv2
    import numpy as np

    # 이미지 로드
    image = cv2.imread(image_path)

    # 이미지 크기 계산
    height, width = image.shape[:2]

    # 이미지를 수평으로 반으로 나눔
    left_side = image[:, :width//2]
    right_side = image[:, width//2:]

    # 왼쪽 부분을 살짝 아래로 내림 (예: 50 픽셀)
    shift_up = 35
    M = np.float32([[1, 0, 0], [0, 1, shift_up]])
    left_shifted = cv2.warpAffine(left_side, M, (left_side.shape[1], left_side.shape[0]))

    # 변환된 이미지를 다시 합침
    shifted_height = left_shifted.shape[0]
    combined = np.zeros((shifted_height, width, 3), dtype=np.uint8)
    combined[:height, :width//2] = left_shifted
    combined[:height, width//2:] = right_side

    # 결과 이미지
    cv2.imwrite('train/redbox_image/sliced_image.jpg', combined)
    return None


In [223]:
image_path = '/Users/402-08/train/img73.jpg'
slice_image(image_path)